# Semantic Search with Recommendations


## Setup

In [2]:
import graphdatascience
graphdatascience.__version__

'1.8'

In [49]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
from langchain.embeddings import OpenAIEmbeddings, BedrockEmbeddings, SentenceTransformerEmbeddings

In [4]:
load_dotenv('.env', override=True)

True

In [5]:
def load_embedding_model(embedding_model_name: str):
    if embedding_model_name == "openai":
        embeddings = OpenAIEmbeddings()
        dimension = 1536
    elif embedding_model_name == "aws":
        embeddings = BedrockEmbeddings()
        dimension = 1536
    else:
        embeddings = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2", cache_folder="/embedding_model"
        )
        dimension = 384
    return embeddings, dimension

In [6]:
embedding_model, dimension = load_embedding_model(os.getenv('EMBEDDING_MODEL'))

In [7]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

In [8]:
gds.version()

'2.5.1+39'

## Simple Vector Search

Basic product search using vector index

In [39]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'winter sweater with zipper'

In [40]:
query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

query vector length: 1536
query vector sample: [-0.015801964355454827, -0.016015505446318423, -0.0007494756150188125, -0.020686693525895174, -0.025878386617468073, 0.013920143271106442, -0.009462494419748136, -0.010056402551914138, 0.0030112485454235807, -0.01943214613632958]


In [41]:
res_df = gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
    ORDER BY score DESC
''', params={'queryVector': query_vector})

In [42]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
res_df

User Search Prompt: "winter sweater with zipper"


Search Results:



,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand...",0.933456
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up col...",0.931803
2,Ben zip hoodie,Sweater,Knitwear,Fine-knit jacket in a soft viscose blend with ...,0.931028
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,Jumper in a soft rib knit with a high stand-up...,0.930956
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional ...",0.930461
5,Southern Sweater,Sweater,Knitwear,Jumper in a soft rib knit with pointelle detai...,0.929188
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ri...",0.928707
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton b...",0.928681
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtl...",0.927683
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribb...",0.927475


## Recommendation Based on Search Prompt

In [43]:
search_res_df = gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
    ORDER BY score DESC
''', params={'queryVector': query_vector})

In [44]:
recommendation_res_df =gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 5, $queryVector)
YIELD node AS search_res_product, score AS search_score
MATCH(search_res_product)<-[:VARIANT_OF]-(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)-[:VARIANT_OF]-(p)
WITH p.prodName AS prodName,
    p.productTypeName AS productTypeName,
    p.garmentGroupName AS garmentGroupName,
    p.detailDesc AS detailDesc,
    sum(s.score) AS aggScore,
    sum(search_score) AS search_score
RETURN prodName, productTypeName, garmentGroupName, search_score*aggScore AS score, detailDesc
    ORDER BY aggScore DESC LIMIT 10
''', params={'queryVector': query_vector})

In [45]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
display(search_res_df)
print('User May Also Be Interested In:\n')
display(recommendation_res_df)

User Search Prompt: "winter sweater with zipper"


Search Results:



,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand...",0.933456
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up col...",0.931803
2,Ben zip hoodie,Sweater,Knitwear,Fine-knit jacket in a soft viscose blend with ...,0.931028
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,Jumper in a soft rib knit with a high stand-up...,0.930956
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional ...",0.930461
5,Southern Sweater,Sweater,Knitwear,Jumper in a soft rib knit with pointelle detai...,0.929188
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ri...",0.928707
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton b...",0.928681
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtl...",0.927683
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribb...",0.927475


User May Also Be Interested In:



,prodName,productTypeName,garmentGroupName,score,detailDesc
0,Niffler Trousers,Trousers,Woven/Jersey/Knitted mix Baby,12.908029,"Pull-on trousers in washed, stretch twill with..."
1,SLIM PULL ON 5 PKT,Trousers,Trousers Denim,12.571324,"Slim-fit, pull-on jeans in washed, superstretc..."
2,Devon basic sweater,Sweater,Knitwear,12.563571,Jumper in a soft knit with visible seams at th...
3,Sunspot Seamless Crop Top,Vest top,Jersey Fancy,11.540532,"Short, fitted sports top with a racer back and..."
4,TVP Slim denim trousers,Trousers,Woven/Jersey/Knitted mix Baby,3.228393,"Slim-fit jeans in washed, supersoft, stretch d..."
5,Brigita,Sweater,Knitwear,3.220222,"Jumper in a soft, cable knit containing some w..."
6,Don sweater,Sweater,Knitwear,3.202113,Jumper in a cable-knit cotton blend with long ...
7,Cool Kite anklet pk,Other accessories,Accessories,3.189231,Shiny metal chain anklets. Adjustable length 2...
8,Robbie Tee 9.99:-,T-shirt,Jersey Fancy,3.188948,T-shirt in soft cotton with contrasting colour...
9,LW (K) CATRIN consc cardi,Cardigan,"Under-, Nightwear",3.185877,Short cardigan in a soft knit containing some ...


## Personalized Recommendations

In [51]:
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"

In [61]:
MAX_DATE = datetime(year=2019, month=1, day=1)

### Example Purchase History

In [80]:
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a)-[:VARIANT_OF]->(p:Product) WHERE r.tDat > date($cutOffDate)
        RETURN a.articleId AS articleId,
        a.prodName AS prodName,
        r.tDat AS purchaseDate,
        a.productTypeName AS productTypeName,
        p.detailDesc AS detailDesc
        ORDER BY purchaseDate DESC
''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID, 'queryVector': query_vector})

,articleId,prodName,purchaseDate,productTypeName,detailDesc
0,753724004,Rosemary,2019-08-05,Dress,Short dress in woven fabric with 3/4-length sl...
1,733027002,Tove,2019-08-05,Top,Short top in soft cotton jersey with a round n...
2,713577001,Malte r-neck,2019-06-27,Sweater,"Jumper in soft, patterned, fine-knit cotton wi..."
3,731142001,Lead Superskinny,2019-06-27,Trousers,Chinos in stretch twill with a zip fly and but...
4,606711001,Rylee flatform,2019-06-22,Heeled sandals,"Sandals with imitation suede straps, an elasti..."
5,642498007,Bubble Bum Bandeau (1),2019-06-22,Bikini top,Fully lined bandeau bikini top with padded cup...
6,752193001,Banks,2019-06-22,Hoodie,Long-sleeved top in sweatshirt fabric made fro...
7,687016004,DORIS CREW,2019-06-22,Sweater,Top in sweatshirt fabric with a motif on the f...
8,688537001,Simple as that Cheeky Tanga,2019-06-22,Swimwear bottom,Fully lined bikini bottoms with a mid waist an...
9,598806011,Dixie tee,2019-02-08,T-shirt,Short top in soft cotton jersey with short sle...


### Produce Personalized Product Recommendations

In [77]:
personalized_res_df = gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.tDat > date($cutOffDate)
    WITH a0
    MATCH(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)-[:VARIANT_OF]->(p:Product)
    WITH p, sum(s.score) AS aggRecScore
    WITH p, aggRecScore, gds.similarity.cosine($queryVector, p.textEmbedding) AS cosineSimilarity
    RETURN p.productCode AS productCode,
        aggRecScore,
        cosineSimilarity as searchScore,
        p.productTypeName AS productType,
        p.prodName AS name,
        p.detailDesc AS description
        ORDER BY searchScore DESC LIMIT 10
''', params = {'cutOffDate':str(MAX_DATE - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID, 'queryVector': query_vector})

In [78]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
display(search_res_df)
print('User May Also Be Interested In:\n')
display(personalized_res_df)

User Search Prompt: "winter sweater with zipper"


Search Results:



,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand...",0.933456
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up col...",0.931803
2,Ben zip hoodie,Sweater,Knitwear,Fine-knit jacket in a soft viscose blend with ...,0.931028
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,Jumper in a soft rib knit with a high stand-up...,0.930956
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional ...",0.930461
5,Southern Sweater,Sweater,Knitwear,Jumper in a soft rib knit with pointelle detai...,0.929188
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ri...",0.928707
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton b...",0.928681
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtl...",0.927683
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribb...",0.927475


User May Also Be Interested In:



,productCode,aggRecScore,searchScore,productType,name,description
0,934053,1.537385,0.838341,Sweater,Chain,Jumper in a soft knit containing some wool. Ri...
1,656401,11.331780,0.834547,Sweater,PASTRY SWEATER,"Jumper in soft, textured-knit cotton with long..."
2,244267,1.520123,0.825770,Sweater,Silver lake,Purl-knit jumper in a cotton blend with a slig...
3,687016,1.746859,0.824452,Sweater,DORIS CREW,Top in sweatshirt fabric with a motif on the f...
4,713577,9.166964,0.822703,Sweater,Malte r-neck,"Jumper in soft, patterned, fine-knit cotton wi..."
5,662328,3.064164,0.807520,Blouse,Survivor,Straight-cut blouse in a crêpe weave with a co...
6,705754,0.802929,0.797755,Night gown,Klara n-slip (J),Fitted nightdress in soft jersey with a lace-t...
7,569974,22.594693,0.796680,Hoodie,DONT USE ROLAND HOOD,Top in sweatshirt fabric with a lined drawstri...
8,691890,1.596616,0.796419,Dress,Irma dress,Short dress in a patterned crêpe weave with a ...
9,630316,0.799703,0.795372,Blouse,Shay blouse,Wide blouse in a soft weave with gathered deta...
